# Scraping ENGLISH comments with from Reddit

In [10]:
import praw
import pandas as pd


# Acessing the reddit api


reddit = praw.Reddit(client_id="_W4jzODMFlagJQ",#my client id
                     client_secret="buJjkiyaGbj0-Yj6BcjsHUUwv0OhCw",  #your client secret
                     user_agent="pfe-crawler", #user agent name
                     username = "shaddow99",     # your reddit username
                     password = "Wll1616-")     # your reddit password

In [11]:
sub = ['iPhone12']  # make a list of subreddits you want to scrape the data from

for s in sub:
    subreddit = reddit.subreddit(s)   # Chosing the subreddit


########################################
#   CREATING DICTIONARY TO STORE THE DATA WHICH WILL BE CONVERTED TO A DATAFRAME
########################################

#   NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS AS WE WILL 
#   BE CAPTURING ALL IDS THAT COME IN OUR WAY

# SCRAPING CAN BE DONE VIA VARIOUS STRATEGIES {HOT,TOP,etc} we will go with keyword strategy i.e using search a keyword
    query = [' ']

    for item in query:
        post_dict = {
            "title" : [],
            "score" : [],
            "id" : [],
            "url" : [],
            "comms_num": [],
            "created" : [],
            "body" : []
        }
        comments_dict = {
            "comment_id" : [],
            "comment_parent_id" : [],
            "comment_body" : [],
            "comment_link_id" : []
        }
        for submission in subreddit.search(query,sort = "top",limit = 1):
            post_dict["title"].append(submission.title)
            post_dict["score"].append(submission.title)
            post_dict["id"].append(submission.title)
            post_dict["url"].append(submission.title)
            post_dict["comms_num"].append(submission.title)
            post_dict["created"].append(submission.title)
            post_dict["body"].append(submission.title)
            
            ##### Acessing comments on the post
            submission.comments.replace_more(limit = 1)
            for comment in submission.comments.list():
                comments_dict["comment_id"].append(comment.id)
                comments_dict["comment_parent_id"].append(comment.parent_id)
                comments_dict["comment_body"].append(comment.body)
                comments_dict["comment_link_id"].append(comment.link_id)
        
        post_comments = pd.DataFrame(comments_dict)

        post_comments.to_csv("../input/"+s+"_comments_"+"subreddit.csv")
        post_data = pd.DataFrame(post_dict)
        post_data.to_csv("../input/"+s+"_"+"subreddit.csv")

In [12]:
import requests
from datetime import datetime
import traceback
import time
import json
import sys

username = ""  # put the username you want to download in the quotes
subreddit = "iPhone12"  # put the subreddit you want to download in the quotes
# leave either one blank to download an entire user's or subreddit's history
# or fill in both to download a specific users history from a specific subreddit

filter_string = None
if username == "" and subreddit == "":
	print("Fill in either username or subreddit")
	sys.exit(0)
elif username == "" and subreddit != "":
	filter_string = f"subreddit={subreddit}"
elif username != "" and subreddit == "":
	filter_string = f"author={username}"
else:
	filter_string = f"author={username}&subreddit={subreddit}"

url = "https://api.pushshift.io/reddit/{}/search?limit=1000&sort=desc&{}&before="

start_time = datetime.utcnow()


def downloadFromUrl(filename, object_type):
	print(f"Saving {object_type}s to {filename}")

	count = 0
	handle = open(filename, 'w')
	previous_epoch = int(start_time.timestamp())
	while True:
		new_url = url.format(object_type, filter_string)+str(previous_epoch)
		json_text = requests.get(new_url, headers={'User-Agent': "Post downloader by /u/Watchful1"})
		time.sleep(1)  # pushshift has a rate limit, if we send requests too fast it will start returning error messages
		try:
			json_data = json_text.json()
		except json.decoder.JSONDecodeError:
			time.sleep(1)
			continue

		if 'data' not in json_data:
			break
		objects = json_data['data']
		if len(objects) == 0:
			break

		for object in objects:
			previous_epoch = object['created_utc'] - 1
			count += 1
			if object_type == 'comment':
				try:
					handle.write(str(object['score']))
					handle.write(" : ")
					handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
					handle.write("\n")
					handle.write(object['body'].encode(encoding='ascii', errors='ignore').decode())
					handle.write("\n-------------------------------\n")
				except Exception as err:
					print(f"Couldn't print comment: https://www.reddit.com{object['permalink']}")
					print(traceback.format_exc())
			elif object_type == 'submission':
				if object['is_self']:
					if 'selftext' not in object:
						continue
					try:
						handle.write(str(object['score']))
						handle.write(" : ")
						handle.write(datetime.fromtimestamp(object['created_utc']).strftime("%Y-%m-%d"))
						handle.write("\n")
						handle.write(object['selftext'].encode(encoding='ascii', errors='ignore').decode())
						handle.write("\n-------------------------------\n")
					except Exception as err:
						print(f"Couldn't print post: {object['url']}")
						print(traceback.format_exc())

		print("Saved {} {}s through {}".format(count, object_type, datetime.fromtimestamp(previous_epoch).strftime("%Y-%m-%d")))

	print(f"Saved {count} {object_type}s")
	handle.close()


downloadFromUrl("posts.txt", "submission")
downloadFromUrl("comments.txt", "comment")


Saving submissions to posts.txt
Saved 100 submissions through 2021-03-05
Saved 200 submissions through 2021-02-23
Saved 300 submissions through 2021-02-17
Saved 400 submissions through 2021-02-11
Saved 500 submissions through 2021-02-02
Saved 600 submissions through 2021-01-29
Saved 700 submissions through 2021-01-24
Saved 800 submissions through 2021-01-20
Saved 900 submissions through 2021-01-16
Saved 1000 submissions through 2021-01-12
Saved 1100 submissions through 2021-01-08
Saved 1200 submissions through 2021-01-05
Saved 1300 submissions through 2021-01-02
Saved 1400 submissions through 2020-12-29
Saved 1500 submissions through 2020-12-26
Saved 1600 submissions through 2020-12-22
Saved 1700 submissions through 2020-12-19
Saved 1800 submissions through 2020-12-16
Saved 1900 submissions through 2020-12-13
Saved 2000 submissions through 2020-12-09
Saved 2100 submissions through 2020-12-06
Saved 2200 submissions through 2020-12-03
Saved 2300 submissions through 2020-11-30
Saved 2400 